In [1]:
!pip3 install kaggle

In [2]:
!mkdir .kaggle

In [3]:
!mkdir ~/.kaggle/

In [4]:
import json
token = {"username":"sudeeprajput21","key":"1766481fd53783ab03477079054a9b6c"}
with open('/content/.kaggle/kaggle.json','w') as file:
  json.dump(token, file)

In [5]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [6]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [7]:
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d srbhshinde/flickr8k-sau

100% 2.07G/2.08G [00:39<00:00, 43.9MB/s]
100% 2.08G/2.08G [00:39<00:00, 56.6MB/s]


In [9]:
!kaggle datasets download -d yliu9999/glove6b50d

 96% 65.0M/67.7M [00:02<00:00, 20.2MB/s]
100% 67.7M/67.7M [00:02<00:00, 29.9MB/s]


In [10]:
!cd /content/{/content}/datasets 

In [11]:
!unzip /content/{/content}/datasets/yliu9999/glove6b50d/glove6b50d.zip

Archive:  /content/{/content}/datasets/yliu9999/glove6b50d/glove6b50d.zip
  inflating: glove.6B.50d.txt        


In [12]:
!unzip /content/{/content}/datasets/srbhshinde/flickr8k-sau/flickr8k-sau.zip

Streaming output truncated to the last 5000 lines.
  inflating: flickr8k-sau/Flickr_Data/Images/2844747252_64567cf14a.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2844846111_8c1cbfc75d.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2844963839_ff09cdb81f.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2845246160_d0d1bbd6f0.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2845691057_d4ab89d889.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2845845721_d0bc113ff7.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2846037553_1a1de50709.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2846785268_904c5fcf9f.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2846843520_b0e6211478.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2847514745_9a35493023.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2847615962_c330bded6e.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2847859796_4d9cb0d31f.jpg  
  inflating: flickr8k-sau/Flickr_Data/Images/2848266893_9693c66275.jpg  


In [13]:
ls

'{'/   flickr8k-sau/   Flickr_Data/   glove.6B.50d.txt   sample_data/


In [14]:
cd Flickr_Data/

/content/Flickr_Data


In [15]:
ls

Flickr_TextData/  Images/


In [16]:
cd ..

/content


In [17]:
flickr_path='Flickr_Data/'

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding,LSTM
from keras.layers.merge import add
import tensorflow as tf

In [19]:
with open(flickr_path + "Flickr_TextData/Flickr8k.token.txt") as filepath:
  captions = filepath.read()
  filepath.close()

In [20]:
captions = captions.split ("\n") [:-1]

In [21]:
len(captions)

40460

In [22]:
captions[:5]

['1000268201_693b08cb0e.jpg#0\tA child in a pink dress is climbing up a set of stairs in an entry way .',
 '1000268201_693b08cb0e.jpg#1\tA girl going into a wooden building .',
 '1000268201_693b08cb0e.jpg#2\tA little girl climbing into a wooden playhouse .',
 '1000268201_693b08cb0e.jpg#3\tA little girl climbing the stairs to her playhouse .',
 '1000268201_693b08cb0e.jpg#4\tA little girl in a pink dress going into a wooden cabin .']

In [23]:
# Creating a dictionary where key is 'img_name' & value is list of captions 

descriptions = {}

for element in captions:
  img_to_cap = element.split("\t")
  img_name = img_to_cap[0].split(".")[0]
  cap = img_to_cap[1]

  if descriptions.get(img_name) == None:
    descriptions[img_name] = []

  descriptions[img_name].append(cap)

In [24]:
descriptions['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

**Data Cleaning**

In [25]:
# Lower each word, remove punctuations, remove words lessthan length 1

def clean_text(sample):
  sample = sample.lower()

  sample = re.sub("[^a-z]+"," ", sample)

  sample = sample.split()

  sample = [s for s in sample if len(s)>1]

  sample = " ".join(sample)

  return sample

In [26]:
clean_text ("I am Sudeep 21 03 rAjput @1996")

'am sudeep rajput'

In [27]:
# Modify all the captions (cleaned captions)

for key, desc_list in descriptions.items():
  for i in range (len(desc_list)):
    desc_list[i] = clean_text(desc_list[i])

In [28]:
descriptions ['1000268201_693b08cb0e']

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [29]:
# Writing clean descriptions to .txt file

f = open("descriptions.txt","w")
f.write(str(descriptions))
f.close()

In [30]:
# Reading descriptions file

f = open("descriptions.txt","r")
descriptions = f.read()
f.close()

json_acceptable_string = descriptions.replace("'","\"")
descriptions = json.loads(json_acceptable_string)

In [31]:
descriptions

{'1000268201_693b08cb0e': ['child in pink dress is climbing up set of stairs in an entry way',
  'girl going into wooden building',
  'little girl climbing into wooden playhouse',
  'little girl climbing the stairs to her playhouse',
  'little girl in pink dress going into wooden cabin'],
 '1001773457_577c3a7d70': ['black dog and spotted dog are fighting',
  'black dog and tri colored dog playing with each other on the road',
  'black dog and white dog with brown spots are staring at each other in the street',
  'two dogs of different breeds looking at each other on the road',
  'two dogs on pavement moving toward each other'],
 '1002674143_1b742ab4b8': ['little girl covered in paint sits in front of painted rainbow with her hands in bowl',
  'little girl is sitting in front of large painted rainbow',
  'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
  'there is girl with pigtails sitting in front of rainbow painting',
  'young girl with p

In [32]:
# Finding unique vocabulary (finding how many unique words are present in 40K captions)

vocabulary = set()

for key in descriptions.keys():
  [vocabulary.update(i.split()) for i in descriptions[key]]

print('Vocabulary Size:%d' %len(vocabulary))







Vocabulary Size:8424


In [33]:
# Write all words in description dictionary

all_vocab = []

for key in descriptions.keys():
  [all_vocab.append(i) for des in descriptions[key] for i in des.split()]

print ('Vocabulary Size:%d' %len(all_vocab))
print (all_vocab[:15])



Vocabulary Size:373837
['child', 'in', 'pink', 'dress', 'is', 'climbing', 'up', 'set', 'of', 'stairs', 'in', 'an', 'entry', 'way', 'girl']


In [34]:
# Count the frequency of each word, sort them & discard the words having frequency lesser than the threshold value

import collections

counter = collections.Counter(all_vocab)

dict_ = dict(counter)

threshold_value = 10

sorted_dict = sorted(dict_.items(), reverse = True, key = lambda x: x[1])
sorted_dict = [x for x in sorted_dict if x[1] > threshold_value]
all_vocab = [x[0] for x in sorted_dict]



In [35]:
len(all_vocab)

1845

In [36]:
# Train image file

f = open (flickr_path + "Flickr_TextData/Flickr_8k.trainImages.txt")
train = f.read()
f.close()

In [37]:
train

'2513260012_03d33305cf.jpg\n2903617548_d3e38d7f88.jpg\n3338291921_fe7ae0c8f8.jpg\n488416045_1c6d903fe0.jpg\n2644326817_8f45080b87.jpg\n218342358_1755a9cce1.jpg\n2501968935_02f2cd8079.jpg\n2699342860_5288e203ea.jpg\n2638369467_8fc251595b.jpg\n2926786902_815a99a154.jpg\n2851304910_b5721199bc.jpg\n3423802527_94bd2b23b0.jpg\n3356369156_074750c6cc.jpg\n2294598473_40637b5c04.jpg\n1191338263_a4fa073154.jpg\n2380765956_6313d8cae3.jpg\n3197891333_b1b0fd1702.jpg\n3119887967_271a097464.jpg\n2276499757_b44dc6f8ce.jpg\n2506892928_7e79bec613.jpg\n2187222896_c206d63396.jpg\n2826769554_85c90864c9.jpg\n3097196395_ec06075389.jpg\n3603116579_4a28a932e2.jpg\n3339263085_6db9fd0981.jpg\n2532262109_87429a2cae.jpg\n2076906555_c20dc082db.jpg\n2502007071_82a8c639cf.jpg\n3113769557_9edbb8275c.jpg\n3325974730_3ee192e4ff.jpg\n1655781989_b15ab4cbff.jpg\n1662261486_db967930de.jpg\n2410562803_56ec09f41c.jpg\n2469498117_b4543e1460.jpg\n69710415_5c2bfb1058.jpg\n3414734842_beb543f400.jpg\n3006217970_90b42e6b27.jpg\n2192

In [38]:
train = [e.split(".")[0] for e in train.split("\n")[:-1]]

In [39]:
# Test image file

f = open (flickr_path + "Flickr_TextData/Flickr_8k.testImages.txt")
test = f.read()
f.close()

In [40]:
test

'3385593926_d3e9c21170.jpg\n2677656448_6b7e7702af.jpg\n311146855_0b65fdb169.jpg\n1258913059_07c613f7ff.jpg\n241347760_d44c8d3a01.jpg\n2654514044_a70a6e2c21.jpg\n2339106348_2df90aa6a9.jpg\n256085101_2c2617c5d0.jpg\n280706862_14c30d734a.jpg\n3072172967_630e9c69d0.jpg\n3482062809_3b694322c4.jpg\n1167669558_87a8a467d6.jpg\n2847615962_c330bded6e.jpg\n3344233740_c010378da7.jpg\n2435685480_a79d42e564.jpg\n3110649716_c17e14670e.jpg\n2511019188_ca71775f2d.jpg\n2521770311_3086ca90de.jpg\n2723477522_d89f5ac62b.jpg\n2218609886_892dcd6915.jpg\n3745451546_fc8ec70cbd.jpg\n2844018783_524b08e5aa.jpg\n3100251515_c68027cc22.jpg\n2207244634_1db1a1890b.jpg\n2943023421_e297f05e11.jpg\n3286822339_5535af6b93.jpg\n2479652566_8f9fac8af5.jpg\n1394368714_3bc7c19969.jpg\n872622575_ba1d3632cc.jpg\n2309860995_c2e2a0feeb.jpg\n241347204_007d83e252.jpg\n3502343542_f9b46688e5.jpg\n757332692_6866ae545c.jpg\n2748729903_3c7c920c4d.jpg\n494792770_2c5f767ac0.jpg\n3213992947_3f3f967a9f.jpg\n2295750198_6d152d7ceb.jpg\n23588980

In [41]:
test = [e.split(".")[0] for e in test.split("\n")[:-1]]

In [42]:
# Create train descriptions dictionary which will be similar to earlier one but having only train samples
# add startseq + endseq

train_descriptions = {}

for t in train:
  train_descriptions[t] = []

  for cap in descriptions[t]:
    cap_to_append = "startseq " + cap + " endseq"
    train_descriptions[t].append (cap_to_append)

In [43]:
# Mapping descriptions to images

train_descriptions['1000268201_693b08cb0e']

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

In [44]:
images = flickr_path = "Images/"

**Model for preprocessing**

In [45]:
model = ResNet50(weights="imagenet", input_shape=(224,224,3))

102973440/102967424 [==============================] - 1s 0us/step


In [46]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [47]:
# Create a new model by removing the last layer(output layer of 1000 classes) fro resnet50

model_new = Model(model.input, model.layers[-2].output)

In [48]:
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [49]:
def preprocess_image(img):
  img = image.load_img(img, target_size=(224,224))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  return img


In [50]:
def encode_image(img):
  img = preprocess_image(img)
  feature_vector = model_new.predict(img)
  feature_vector = feature_vector.reshape(feature_vector.shape[1])
  return feature_vector

In [ ]:
# Encoding each image in train set using Resnet

encoding_train = {}

for index, img in enumerate(train):

  img = "/content/Flickr_Data/Images/{}.jpg".format(train[index])
  encoding_train[img[len(images):]] = encode_image(img)

  if index%100==0:
        print ("Encoding image-" + str(index))

Encoding image-0
Encoding image-100
Encoding image-200
Encoding image-300
Encoding image-400
Encoding image-500
Encoding image-600
Encoding image-700
Encoding image-800
Encoding image-900
Encoding image-1000
Encoding image-1100
Encoding image-1200
Encoding image-1300
Encoding image-1400
Encoding image-1500
Encoding image-1600
Encoding image-1700
Encoding image-1800
Encoding image-1900
Encoding image-2000
Encoding image-2100
Encoding image-2200
Encoding image-2300
Encoding image-2400
Encoding image-2500
Encoding image-2600
Encoding image-2700
Encoding image-2800
Encoding image-2900
Encoding image-3000
Encoding image-3100
Encoding image-3200
Encoding image-3300
Encoding image-3400
Encoding image-3500
Encoding image-3600
Encoding image-3700
Encoding image-3800
Encoding image-3900


In [ ]:
# Save the train features to disk

with open ("./encoded_train_images.pkl", "wb") as encoded_pickle:
   pickle.dump(encoding_train, encoded_pickle)

In [ ]:
# Encoding test images

encoding_test = {}

for index, img in enumerate(test):

  img = "/content/Flickr_Data/Images/{}.jpg".format(test[index])
  encoding_test[img[len(images):]] = encode_image(img)

  if index%100==0:
        print ("Encoding image-" + str(index))

In [ ]:
# Save the bottleneck train features to disk

with open ("./encoded_test_images.pkl", "wb") as encoded_pickle:
  pickle.dump(encoding_test, encoded_pickle)

In [ ]:
# Load the train images features from disk

with open ("./encoded_train_images.pkl", "rb") as encoded_pickle:
  encoding_train=pickle.load(encoded_pickle)

In [ ]:
encoding_train['2513260012_03d33305cf.jpg']

In [ ]:
# Load the test image features from disk

with open ("./encoded_test_images.pkl", "rb") as encoded_pickle:
  encoding_test = pickle.load(encoded_pickle)

In [ ]:
# word_to_index is mapping between each unique word in all_vocab to int value & index_to_word is vice versa.

index = 1
word_to_index = {}
index_to_word = {}


for e in all_vocab:
  word_to_index[e] = index
  index_to_word[index] = e
  index+= 1 


In [ ]:
word_to_index ['above']

In [ ]:
index_to_word [225]

In [ ]:
len(word_to_index)

In [ ]:
# Adding startseq & endseq

word_to_index ['startseq'] = 1846
word_to_index ['endseq'] = 1847

index_to_word [1846] = 'startseq'
index_to_word [1847] = 'endseq'



In [ ]:
# vocab_size is total vocabulary len +1 because we will append 0's as well

vocab_size = len(index_to_word)+1
print(vocab_size)

In [ ]:
all_captions_len = []

for key in train_descriptions.keys():
    for cap in train_descriptions[key]:
        all_captions_len.append(len(cap.split()))

max_len = max(all_captions_len)
print(max_len)

In [ ]:
def data_generator (train_descriptions, encoding_train, word_to_index, max_len, num_photos_per_batch):

  x1, x2, y=[], [], []
  n = 0

  while True:

    for key, desc_list in train_descriptions.items():
      n += 1

      photo = encoding_train[key+".jpg"]

      for desc in desc_list:

        seq = [word_to_index[word] for word in desc.split() if word in word_to_index]

        for i in range(1,len(seq)):

          in_seq = seq[0:i]
          out_seq = seq[i]

          in_seq = pad_sequences ([in_seq], maxlen = max_len, value = 0, padding = 'post')[0]

          out_seq = to_categorical ([out_seq], num_classes = vocab_size)[0]

          x1.append(photo)
          x2.append(in_seq)
          y.append(out_seq)

      if n == num_photos_per_batch:
        yield [[np.array(x1), np.array(x2)], np.array(y)]
        x1, x2, y = [], [], []
        n=0

In [ ]:
f = open("glove.6B.50d.txt", encoding = 'utf8')
f

In [ ]:
embedding_index = {}

for line in f:
  values = line.split()
  word =  values[0]
  Coefs = np.array(values[1: ], dtype="float")

  embedding_index[word] = Coefs
f.close()

In [ ]:
# Converting words into vector directly - (Embedding Layer Output)

def get_embedding_output():
  emb_dim = 50
  embedding_output = np.zeros((vocab_size, emb_dim))

  for word, idx in word_to_index.items():
    embedding_vector = embedding_index.get(word)
  
    if embedding_vector is not None:
      embedding_output[idx] = embedding_vector
      
  return embedding_output

embedding_output = get_embedding_output()

In [ ]:
embedding_output

In [ ]:
embedding_output.shape

**Defining Model**

Since the input consists of 2 parts, an image vector & partial caption, we cannot use sequential model so we create custom.

In [ ]:
# Image feature extraction model, inputs image feature vector

input_img_feat = Input(shape=(2048))
inp_img1 = Dropout(0.3)(input_img_feat)
inp_img2 = Dense(256, activation='relu')(inp_img1)

In [ ]:
# Partial caption sequence model, inputs captions

input_cap = Input(shape = max_len)
inp_cap1 = Embedding(input_dim= vocab_size, output_dim= 50, mask_zero= True)(input_cap)
inp_cap2 = Dropout(0.3)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

In [ ]:
decoder1 = add([inp_img2,inp_cap3])
decoder2 = Dense(256, activation= 'relu')(decoder1)
outputs = Dense(vocab_size, activation= 'softmax')(decoder2)

# Merge two networks
model = Model(inputs = [input_img_feat, input_cap], outputs = outputs)

In [ ]:
model.summary()

In [ ]:
model.layers[2].set_weights([embedding_output])
model.layers[2].trainable = False

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam")

In [ ]:
epochs = 20
number_pics_per_batch = 3
steps = len(train_descriptions)

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, encoding_train, word_to_index, max_len, number_pics_per_batch)

    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

    model.save('./model_' + str(i) + '.h5')

In [ ]:
# Taking model from after 19th epoch

model = model("./model_19.h5")

In [ ]:
def predict_caption(photo):
  in_text = "startseq"

  for i in range(max_len):
    sequence = [word_to_index[w] for w in in_text.split() if w in word_to_index]
    sequence = pad_sequences([sequence], maxlen = max_len, padding = 'post')

    ypred = model.predict([photo, sequence])
    ypred = ypred.argmax()
    word = index_to_word[ypred]
    in_text = ' ' + word

    if word = 'endseq':
      break

   final_caption = in_text.split()
   final_caption = final_caption[1 :-1]
   final_caption = ' '.join(final_caption)

   return final_caption

In [ ]:
for i in range(20):
  rn = np.random.randint(0, 1000)
  img_name = list(encoding_test.keys())[rn]
  photo = encoding_test[img_name].reshape((1, 2048))

  i = plt.imread(images + img_name)
  plt.imshow(i)
  plt.axis("off")
  plt.show()

  caption = predict_caption(photo)
  print(caption)